Done by Nahian Tahmin

26th January 2022

iTQAN Intern Assessment

In [1]:
import pandas as pd
import numpy as np

In [2]:
time_format = '%d-%m-%Y %H:%M'
date_format = '%d-%m-%Y'
dataset_name = 'DOTUSDT'

In [3]:
def merge_files(dataset_name):

    #reading csv files
    kline = pd.read_csv('klines/' + dataset_name + '.csv')
    fund_rate = pd.read_csv('funding_rate/Funding Rate History_' + dataset_name + ' Perpetual_2021-02-23.csv')
    fund_rate.rename(columns={"Time": "date"}, inplace = True)
    
    #changing the date column data type and generalizing the format
    kline["date"] =  pd.to_datetime(kline["date"]).dt.strftime(time_format)
    fund_rate["date"] =  pd.to_datetime(fund_rate["date"]).dt.strftime(time_format)
    
    #merging the file data
    merged = pd.merge(kline, fund_rate, on='date', how='left')
    merged = merged.drop('fundingRate', 1)

    #storing in a csv file
    merged.to_csv(r'merged_'+ dataset_name + '.csv',index=False, encoding='utf-8')

In [4]:

merge_files(dataset_name)

C:\Users\Asus\AppData\Local\Temp/ipykernel_8840/1319646483.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  merged = merged.drop('fundingRate', 1)


In [5]:
def subset_merged(dataset_name, start_date, end_date):
    #the end and start dates must be in the format '%Y-%m-%d'
    
    merged_file = pd.read_csv('merged_'+ dataset_name + '.csv')
    
    #changing the string dates to datetime type
    merged_file["date"] = pd.to_datetime(merged_file["date"])
    start_data = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    #selecting the rows that fit in the given date time window frame
    mask = merged_file['date'].dt.date.between(start_data, end_date)

    #extracting the selescted rows
    new_merged = merged_file[mask]
    
    #storing in a csv file
    new_merged.to_csv(r'merged.'+ dataset_name + '.dates.selected.csv',index=False, encoding='utf-8')
    
    

In [6]:
subset_merged(dataset_name, '2020-08-18', '2021-01-01')

c:\users\asus\appdata\local\programs\python\python39\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [7]:
def subset_merged_fr(dataset_name, fund_rate):
    
    #loading the data
    selected_merged = pd.read_csv('merged.'+ dataset_name + '.dates.selected.csv')
    
    #pre-processing funding rate column by removing the NAN values and turning it into numeric type
    selected_merged['Funding Rate'] = selected_merged['Funding Rate'].apply(lambda x: str(x))
    selected_merged['Funding Rate'] = selected_merged['Funding Rate'].apply(lambda x: float(str(x).strip('%')) if not x=='nan' else None)

    #selecting the rows that has funding rate greater or equal to the stated value
    mask = selected_merged['Funding Rate'] >= fund_rate

    #extracting the selescted rows
    new_selected = selected_merged[mask]
    
    #storing in a csv file
    new_selected.to_csv(r'merged.'+ dataset_name + '.dates.selected.funding.rate.csv',index=False, encoding='utf-8')
    
    

In [8]:
subset_merged_fr(dataset_name, 0.0001)

In [9]:
def subset_merged_fr(dataset_name, fund_rate, minutes_to_check):
    
    #loading the data
    selected_merged = pd.read_csv('merged.'+ dataset_name + '.dates.selected.csv')
    selected_merged['date'] = pd.to_datetime(selected_merged['date'])
    
    #pre-processing funding rate column by removing the NAN values and turning it into numeric type
    selected_merged['Funding Rate'] = selected_merged['Funding Rate'].apply(lambda x: str(x))
    selected_merged['Funding Rate'] = selected_merged['Funding Rate'].apply(lambda x: float(str(x).strip('%')) if not x=='nan' else None)

    #selecting the rows that has funding rate greater or equal to the stated value
    mask = selected_merged['Funding Rate'] >= fund_rate

    #extracting the selescted rows
    new_selected_1 = selected_merged[mask]
    
    #declaring empty dataframe to fill with the minutes to check data
    new_selected_2 = pd.DataFrame(columns = list(selected_merged.columns))
    
    for index, row in new_selected_1.iterrows():
        start_date = row['date']
        end_date = row['date'] + pd.Timedelta("" + str(minutes_to_check) + " min")
        
        #the following lines are from the subset_merged function above
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
    
        #selecting the rows that fit in the given date time window frame
        mask = selected_merged['date'].between(start_date, end_date)

        #extracting the selescted rows
        new_merged = selected_merged[mask]
        
        new_selected_2 = new_selected_2.append(new_merged, ignore_index = True)
    
    #storing in a csv file
    new_selected_2.to_csv(r'merged.'+ dataset_name + '.dates.selected.funding.rate.minutes.csv',index=False, encoding='utf-8')
    
    

In [10]:
subset_merged_fr(dataset_name, 0.0001, 5)

In [13]:
def buy_or_sell(dataset_name, start_date, end_date, funding_rate, minutes_to_check):
    #the end and start dates must be in the format '%Y-%m-%d'
    
    #preprocessing the dataset using the appropriate functions
    subset_merged(dataset_name, start_date, end_date)
    subset_merged_fr(dataset_name, funding_rate, minutes_to_check)
    
    final = pd.read_csv('merged.'+ dataset_name + '.dates.selected.funding.rate.minutes.csv')
    
    funding_rate_final = []
    #finding the aveage funding rate for each funding rate value in the minutes to check time frame
    for i in range(0, len(final) - minutes_to_check, minutes_to_check) :
        funding_rate_final.append(np.nanmean(final.loc[i:i+minutes_to_check, 'Funding Rate']))
    
    #adding the funding rate to see if it is long or short
    total_fr = sum(funding_rate_final)

    #printing if the currency should be sold or bought depending on the long/short of funding rate
    if total_fr > 0:       #Long
        print('Buy')
    else:                  #Short
        print('Sell')
        
    

In [14]:
buy_or_sell(dataset_name, '2020-08-18', '2021-01-01', 0.0001, 5)

Buy
